## Appraisal model v.1
appraisal model using xgboost
### Import Libraries and Read preprocessed data

In [1]:
import pandas as pd
import numpy as np
import xgboost as xg
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.metrics import mean_absolute_error as MAE 

In [2]:
df = pd.read_csv('../AppraisalDataPreprocessing/preprocessed_data.csv')
df.head(5)

,car_year,model_year_start,model_year_end,mile,cost,model_2,model_3,model_323,model_BT-50,model_BT-50 PRO,...,color_gray,color_green,color_orange,color_other,color_red,color_silver,color_sky,color_white,color_yellow,car_model
0,-0.436669,-0.930677,-1.150860,0.819832,245000,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Mazda 2 1.5 Sports Maxx Sports Hatchback
1,-1.212501,-0.603686,-1.150860,-0.053048,269000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Mazda 3 2.0 Maxx Sports Hatchback
2,1.114996,0.050294,0.563974,-0.722757,390000,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Mazda 2 1.3 S Leather Sedan
3,1.114996,0.867769,0.563974,-0.045523,650000,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Mazda CX-30 2.0 SP SUV
4,-0.178058,0.050294,0.563974,1.007953,357900,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Mazda 2 1.3 High Connect Sedan


### Appraise cost using xgboost
get mean of each car model for evaluate

In [3]:
df['car_model_mean'] = df['cost'].groupby(df['car_model']).transform(np.mean)

seperate target column from dataframe

In [4]:
X = df.drop(columns=['cost', 'car_model'])
y = df['cost']

splitting data  
train : test = 80 : 20  
stratify by car model

In [5]:
car_model = df['car_model']
df.drop(columns=['car_model'], inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=car_model, random_state=1234)

In [6]:
X_train.drop(columns=['car_model_mean'], inplace=True)
car_model_mean = X_test['car_model_mean']
X_test.drop(columns=['car_model_mean'], inplace=True)

Use xgboost model to appraise the value

In [7]:
xgb_r = xg.XGBRegressor(objective='reg:linear', n_estimators=10, seed=1234)
xgb_r.fit(X_train, y_train)
pred = xgb_r.predict(X_test)

[20:55:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


### Evaluate model

In [8]:
mae = MAE(y_test, pred)
mse = MSE(y_test, pred, squared=True)
rmse = MSE(y_test, pred, squared=False)

print('MAE = ', mae)
print('MSE = ', mse)
print('RMSE = ', rmse)

MAE =  39384.16048547836
MSE =  3878927906.6514335
RMSE =  62281.0397043228


In [9]:
mae = MAE(y_test, car_model_mean)
mse = MSE(y_test, car_model_mean, squared=True)
rmse = MSE(y_test, car_model_mean, squared=False)

print('MAE = ', mae)
print('MSE = ', mse)
print('RMSE = ', rmse)

MAE =  52904.42943926512
MSE =  5953253163.0976515
RMSE =  77157.32734548062
